In [1]:
!python -m pip install psycopg2-binary apyori

You should consider upgrading via the 'C:\Users\abcis\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2 as pg
from apyori import apriori
%matplotlib inline
import getpass

In [2]:
password = getpass.getpass()

In [5]:
def pg_connect():
    pg_user = "bgwoiervutgzta"
    pg_pwd = password
    pg_host = "ec2-44-193-228-249.compute-1.amazonaws.com"
    pg_port = 5432
    pg_dbname = "d18l93hamvmtkj"
    return pg.connect(f"dbname='{pg_dbname}' user='{pg_user}' host='{pg_host}' port='{pg_port}' password='{pg_pwd}'")

In [6]:
sql = "SELECT * FROM users join signup_metadata using(user_id)"
df = pd.read_sql(sql, con=pg_connect())


In [7]:
pd.set_option('display.max_columns', None)

In [8]:
df.head(3)

,user_id,user_name,user_email,user_password,user_phone,user_is_admin,signup_md_id,signup_md_device_id,signup_md_paste_count,signup_md_start_date,signup_md_end_date,signup_md_timezone,signup_md_ip,signup_md_os,signup_md_screen_width,signup_md_screen_height,signup_md_device_name,signup_md_browser,signup_md_cpu_cores,signup_md_gpu_name
0,4,Samuel Oliveira,samuel.oliveira@email.com,senhanova,(12) 89887-7626,None,3,98a6045f83298d284b5d20171df903a9,0,2021-09-19 17:16:57.803,2021-09-19 17:17:33.873,America/Sao_Paulo,177.170.125.38,Linux,1366,768,not collected,firefox,8.0,radeon r9 200
1,5,Gabriel Francisco,gabriel.francisco@email.com,senhanova,(12) 97685-5442,None,4,98a6045f83298d284b5d20171df903a9,0,2021-09-19 17:18:26.971,2021-09-19 17:19:07.983,America/Sao_Paulo,177.170.125.38,Linux,1366,768,not collected,firefox,8.0,radeon r9 200
2,6,Teste Aaaa Uau,teste@aaaaaaaa.com.br,1234567890,(12) 98811-2312,None,5,6218ae75e8a1180faf71bbee9b10961d,0,2021-09-19 19:06:20.627,2021-09-19 19:06:20.832,America/Sao_Paulo,,Windows 10,1920,1080,not collected,chrome,8.0,amd radeon vega 8 mobile


In [9]:
df.drop(columns=['signup_md_end_date','signup_md_start_date','signup_md_paste_count','user_phone','user_email','user_name','user_is_admin','signup_md_device_name', 'signup_md_cpu_cores','user_password'], inplace=True)

In [10]:
df.shape

(14, 10)

In [11]:
records = []
for i in range(0, 10): records.append([str(df.values[i,j]) for j in range(0, 10)])

In [12]:
records

[['4',
  '3',
  '98a6045f83298d284b5d20171df903a9',
  'America/Sao_Paulo',
  '177.170.125.38',
  'Linux',
  '1366',
  '768',
  'firefox',
  'radeon r9 200'],
 ['5',
  '4',
  '98a6045f83298d284b5d20171df903a9',
  'America/Sao_Paulo',
  '177.170.125.38',
  'Linux',
  '1366',
  '768',
  'firefox',
  'radeon r9 200'],
 ['6',
  '5',
  '6218ae75e8a1180faf71bbee9b10961d',
  'America/Sao_Paulo',
  '',
  'Windows 10',
  '1920',
  '1080',
  'chrome',
  'amd radeon vega 8 mobile'],
 ['8',
  '6',
  '6149ed31bcc27ca1262dd28a3f3f4ba1',
  'America/Sao_Paulo',
  '187.34.105.191',
  'Linux',
  '1366',
  '768',
  'chrome',
  'amd radeon vega 8 mobile'],
 ['9',
  '7',
  '6149ed31bcc27ca1262dd28a3f3f4ba1',
  'America/Sao_Paulo',
  '187.34.105.191',
  'Linux',
  '1366',
  '768',
  'chrome',
  'amd radeon vega 8 mobile'],
 ['10',
  '8',
  '6149ed31bcc27ca1262dd28a3f3f4ba1',
  'America/Sao_Paulo',
  '187.34.105.191',
  'Linux',
  '1366',
  '768',
  'chrome',
  'radeon r9 200'],
 ['11',
  '9',
  '98a6045f8329

In [13]:
regras = apriori(records, min_length = 2, min_support = 0.2, min_confidence = 0.2, min_lift = 3)

In [14]:
result = list(regras)

In [15]:
result[0]

RelationRecord(items=frozenset({'1080', '1920'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'1080'}), items_add=frozenset({'1920'}), confidence=1.0, lift=3.3333333333333335), OrderedStatistic(items_base=frozenset({'1920'}), items_add=frozenset({'1080'}), confidence=1.0, lift=3.3333333333333335)])

In [16]:
print(len(result))

538


In [17]:
r = pd.DataFrame(columns=('Items','Antecendete','Consequente','Suporte','Confiança','Lift'))

support=[]
confidence = []
lift =[]
items = []
antecedent = []
consequent = []

for RelationRecord in result:
    for ordered_stat in RelationRecord.ordered_statistics:
        support.append(round(RelationRecord.support,4))
        items.append(RelationRecord.items)
        antecedent.append(ordered_stat.items_base)
        consequent.append(ordered_stat.items_add)
        confidence.append(round(ordered_stat.confidence,4))
        lift.append(round(ordered_stat.lift,4))

r['Items'] = list(map(set,items))
r['Antecendete'] = list(map(set,antecedent))
r['Consequente'] = list(map(set,consequent))
r['Suporte'] = support
r['Confiança'] = confidence
r['Lift'] = lift

r.sort_values(by='Lift', ascending=False, inplace=True)

print(r)

                                                  Items  \
6465  {firefox, 177.170.125.38, radeon r9 200, Linux...   
7493  {America/Sao_Paulo, 1080, Windows 10, chrome, ...   
7515  {America/Sao_Paulo, 1080, Windows 10, chrome, ...   
7514  {America/Sao_Paulo, 1080, Windows 10, chrome, ...   
7513  {America/Sao_Paulo, 1080, Windows 10, chrome, ...   
...                                                 ...   
6981  {America/Sao_Paulo, 6149ed31bcc27ca1262dd28a3f...   
6980  {America/Sao_Paulo, 6149ed31bcc27ca1262dd28a3f...   
6979  {America/Sao_Paulo, 6149ed31bcc27ca1262dd28a3f...   
6978  {America/Sao_Paulo, 6149ed31bcc27ca1262dd28a3f...   
0                                          {1080, 1920}   

                                            Antecendete  \
6465                                           {768, 4}   
7493                {1080, 1920, chrome, 201.46.19.173}   
7515          {chrome, 1920, Windows 10, 201.46.19.173}   
7514   {America/Sao_Paulo, 1920, chrome, 201.46.19.173}

In [18]:
pd.set_option('display.max_rows', None)

In [19]:
r.head(100)

,Items,Antecendete,Consequente,Suporte,Confiança,Lift
6465,"{firefox, 177.170.125.38, radeon r9 200, Linux...","{768, 4}","{radeon r9 200, firefox, Linux, 177.170.125.38}",0.2,1.0,5.0
7493,"{America/Sao_Paulo, 1080, Windows 10, chrome, ...","{1080, 1920, chrome, 201.46.19.173}","{America/Sao_Paulo, Windows 10, 771d19f2dd3554...",0.2,1.0,5.0
7515,"{America/Sao_Paulo, 1080, Windows 10, chrome, ...","{chrome, 1920, Windows 10, 201.46.19.173}","{America/Sao_Paulo, 1080, 771d19f2dd35546be40d...",0.2,1.0,5.0
7514,"{America/Sao_Paulo, 1080, Windows 10, chrome, ...","{America/Sao_Paulo, 1920, chrome, 201.46.19.173}","{771d19f2dd35546be40d4e07bbad1c53, 1080, Windo...",0.2,1.0,5.0
7513,"{America/Sao_Paulo, 1080, Windows 10, chrome, ...","{America/Sao_Paulo, 1920, Windows 10, 201.46.1...","{771d19f2dd35546be40d4e07bbad1c53, 1080, chrome}",0.2,1.0,5.0
7508,"{America/Sao_Paulo, 1080, Windows 10, chrome, ...","{771d19f2dd35546be40d4e07bbad1c53, 1080, Windo...","{America/Sao_Paulo, 1920, 201.46.19.173}",0.2,1.0,5.0
7507,"{America/Sao_Paulo, 1080, Windows 10, chrome, ...","{771d19f2dd35546be40d4e07bbad1c53, 1080, chrom...","{Windows 10, 1920, 201.46.19.173}",0.2,1.0,5.0
7506,"{America/Sao_Paulo, 1080, Windows 10, chrome, ...","{771d19f2dd35546be40d4e07bbad1c53, 1080, Windo...","{1920, chrome, 201.46.19.173}",0.2,1.0,5.0
7505,"{America/Sao_Paulo, 1080, Windows 10, chrome, ...","{1080, Windows 10, chrome, 201.46.19.173}","{America/Sao_Paulo, 771d19f2dd35546be40d4e07bb...",0.2,1.0,5.0
7504,"{America/Sao_Paulo, 1080, Windows 10, chrome, ...","{America/Sao_Paulo, 1080, chrome, 201.46.19.173}","{771d19f2dd35546be40d4e07bbad1c53, Windows 10,...",0.2,1.0,5.0


In [20]:
#r.to_csv('apriori.csv')

In [20]:
ids = frozenset(list(df['signup_md_device_id'].unique()))

In [22]:
#Filtering results
alt = pd.DataFrame(columns=('Items','Antecendete','Consequente','Suporte','Confiança','Lift'))

support=[]
confidence = []
lift =[]
items = []
antecedent = []
consequent = []

for RelationRecord in result:
    for ordered_stat in RelationRecord.ordered_statistics:
        if ordered_stat.items_add.issubset(ids): 
            support.append(round(RelationRecord.support,4))
            items.append(RelationRecord.items)
            antecedent.append(ordered_stat.items_base)
            consequent.append(ordered_stat.items_add)
            confidence.append(round(ordered_stat.confidence,4))
            lift.append(round(ordered_stat.lift,4))

alt['Items'] = list(map(set,items))
alt['Antecendete'] = list(map(set,antecedent))
alt['Consequente'] = list(map(set,consequent))
alt['Suporte'] = support
alt['Confiança'] = confidence
alt['Lift'] = lift

alt.sort_values(by='Lift', ascending=False, inplace=True)

In [23]:
alt.head(10)

,Items,Antecendete,Consequente,Suporte,Confiança,Lift
33,"{771d19f2dd35546be40d4e07bbad1c53, 1080, 1920,...","{1080, 1920, 201.46.19.173}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
195,"{America/Sao_Paulo, Windows 10, chrome, 771d19...","{America/Sao_Paulo, Windows 10, chrome, 1920, ...",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
99,"{America/Sao_Paulo, 1080, chrome, 771d19f2dd35...","{America/Sao_Paulo, 1080, chrome, 201.46.19.173}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
98,"{America/Sao_Paulo, 1080, Windows 10, 771d19f2...","{America/Sao_Paulo, 1080, Windows 10, 201.46.1...",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
94,"{1080, chrome, 771d19f2dd35546be40d4e07bbad1c5...","{1080, 1920, chrome, 201.46.19.173}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
158,"{America/Sao_Paulo, 1080, chrome, 771d19f2dd35...","{America/Sao_Paulo, 1080, chrome, 1920, 201.46...",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
93,"{1080, Windows 10, 771d19f2dd35546be40d4e07bba...","{1080, 1920, Windows 10, 201.46.19.173}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
92,"{America/Sao_Paulo, 1080, 771d19f2dd35546be40d...","{America/Sao_Paulo, 1080, 1920, 201.46.19.173}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
37,"{771d19f2dd35546be40d4e07bbad1c53, 1080, Ameri...","{America/Sao_Paulo, 1080, 201.46.19.173}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0
38,"{771d19f2dd35546be40d4e07bbad1c53, 1080, Windo...","{1080, Windows 10, 201.46.19.173}",{771d19f2dd35546be40d4e07bbad1c53},0.2,1.0,5.0


In [ ]:
alt.to_csv('apriori.csv')